In [13]:
from profilescraper import query_profile
import pandas as pd
import itertools

import os

from glob import glob

import numpy as np

from proxy_utils import proxy_dict, get_proxies

from datetime import datetime, timedelta

import numba

In [3]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

%load_ext autoreload
%autoreload 2

In [4]:
def profiles_to_pandas(profiles):
    userDf = pd.DataFrame(columns=['username', 'location', 'has_location', 'created', 'is_verified', 'total_tweets', 'total_following', 'total_followers', 'total_likes', 'total_moments', 'total_lists', 'has_avatar', 'has_background', 'is_protected', 'profile_modified', 'tweets'])
    tweetDf = pd.DataFrame(columns=['User', 'ID', 'Tweet', 'Time', 'Likes', 'Replies', 'Retweet'])

    for profile in profiles:   
        for tweet in profile.tweets:
            tweetDf = tweetDf.append({'User': profile.username, 'ID': tweet.id, 'Tweet': tweet.text, 'Time': tweet.timestamp, 'Likes': tweet.likes, 'Replies': tweet.replies, 'Retweet': tweet.retweets}, ignore_index=True)

        userDf = userDf.append({'username':profile.username, 'location':profile.location, 'has_location':profile.has_location, 'created':profile.created, 'is_verified':profile.is_verified, 'total_tweets':profile.total_tweets, 'total_following':profile.total_following, 'total_followers':profile.total_followers, 'total_likes':profile.total_likes, 'total_moments':profile.total_moments, 'total_lists':profile.total_lists, 'has_avatar':profile.has_avatar, 'has_background':profile.has_background, 'is_protected':profile.is_protected, 'profile_modified':profile.profile_modified}, ignore_index=True)

    tweetDf = tweetDf.to_csv('data/profiledata/userTweets.csv', index=None, mode='a')
    userDf['username'].to_csv('data/profiledata/extractedUsers.csv', index=None, mode='a')
    userDf.to_csv('data/profiledata/userData.csv', index=None, mode='a')
    
    print("Saved to userTweets.csv and extractedUsers.csv")

In [5]:
def scrape_list(currList, poolsize, proxy, count):
        
    if (len(currList) > 0):
        profiles = query_profile(currList, poolsize=poolsize, proxy=proxy)
        profiles_to_pandas(profiles)

        count += 1
    
    return count

In [6]:
def perform_extraction(coinname, poolsize=20):
    proxies = get_proxies()
    proxySize = len(proxies)
    
    users = list(set(pd.read_csv('data/{}/extracted/combined.csv'.format(coinname), dtype=str)['User']))
    
    try:
        alreadyRead = pd.read_csv('data/profiledata/extractedUsers.csv', header=None)[0]
    except FileNotFoundError:
        logging.info("Already extracted users not found - Starting from a clean slate")
        os.mknod("data/profiledata/extractedUsers.csv")
        alreadyRead = pd.Series()
        
    
    uniqueUsers = list(set(users) - set(alreadyRead))
    
    print("File contains {} data. Scraping for {} after cache".format(len(users), len(uniqueUsers)))
    
    oldi = 0
    count = 0
    
    for i in range(0, len(uniqueUsers), poolsize*5):
        count = scrape_list(uniqueUsers[oldi:i], poolsize=poolsize, proxy=proxies[count], count=count)
        
        if (count >= proxySize):
            count = 0
        
        logging.info("Done {} of {}".format(i, len(uniqueUsers)))
        oldi = i
    
    scrape_list(uniqueUsers[i:], poolsize=poolsize, proxy=None, count=count)

In [6]:
for files in glob(os.getcwd() + "/*"):
    if (os.path.exists(files + "/extracted")):
        print("Extracting for {}".format(files))
        perform_extraction(files.split("/")[-1], poolsize=30)

Extracting for /home/shresthanikesh23/jupyter/CryptoTraderRemote/CryptoTrader/data_utils/twitter_data/ripple


INFO:root:Done 0 of 1378


File contains 44380 data. Scraping for 1378 after cache


INFO:root:Querying https://twitter.com/theefunnytweets
INFO:root:Querying https://twitter.com/MobileStarts
INFO:root:Querying https://twitter.com/CryptoCoin_Inv
INFO:root:Querying https://twitter.com/Bitcoin_Asset
INFO:root:Querying https://twitter.com/webdesignerHKG
INFO:root:Querying https://twitter.com/skyIite
INFO:root:Querying https://twitter.com/OddIySatisfying
INFO:root:Querying https://twitter.com/GratefulDead_
INFO:root:Querying https://twitter.com/ohnotheydidnt
INFO:root:Querying https://twitter.com/TLNAnchorage
INFO:root:Querying https://twitter.com/WiseCicadaNZ
INFO:root:Querying https://twitter.com/love_tenacious
INFO:root:Querying https://twitter.com/BitcoinXP
INFO:root:Querying https://twitter.com/onthewaythrough
INFO:root:Querying https://twitter.com/ruannicolas10
INFO:root:Querying https://twitter.com/GrowPool
INFO:root:Querying https://twitter.com/_iluvmarijuana
INFO:root:Querying https://twitter.com/eBooksLister
INFO:root:Querying https://twitter.com/LACityCollege
IN

KeyboardInterrupt: 

INFO:root:Querying https://twitter.com/BPG_XAU
INFO:root:Querying https://twitter.com/findtrueloveplz
INFO:root:Querying https://twitter.com/TheBitCoinPRO
INFO:root:Querying https://twitter.com/LEDLightCube
INFO:root:Querying https://twitter.com/planetizenjobs
INFO:root:Querying https://twitter.com/Real_JakeGreen
INFO:root:Querying https://twitter.com/bondappetit04
INFO:root:Querying https://twitter.com/WordPress_Cloud
Process ForkPoolWorker-35:
Process ForkPoolWorker-31:
Process ForkPoolWorker-36:
Process ForkPoolWorker-32:
Process ForkPoolWorker-34:
Process ForkPoolWorker-33:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    s

  File "/home/shresthanikesh23/.local/lib/python3.5/site-packages/requests/adapters.py", line 440, in send
    timeout=timeout
  File "/home/shresthanikesh23/.local/lib/python3.5/site-packages/requests/api.py", line 72, in get
    return request('get', url, params=params, **kwargs)
  File "/home/shresthanikesh23/.local/lib/python3.5/site-packages/requests/sessions.py", line 508, in request
    resp = self.send(prep, **send_kwargs)
  File "/home/shresthanikesh23/.local/lib/python3.5/site-packages/urllib3/connection.py", line 284, in connect
    conn = self._new_conn()
  File "/home/shresthanikesh23/.local/lib/python3.5/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
  File "/home/shresthanikesh23/.local/lib/python3.5/site-packages/urllib3/connectionpool.py", line 816, in _prepare_proxy
    conn.connect()
  File "/home/shresthanikesh23/.local/lib/python3.5/site-packages/requests/sessions.py", line 618, in send
    r = adapter.send(request, **

In [7]:
def print_details():
    allUsers = pd.DataFrame()

    for files in glob(os.getcwd() + "/*"):
        if (os.path.exists(files + "/extracted")):
            fname = files + "/extracted/combined.csv"
            tDf = pd.read_csv(fname)
            print("Reading from {}".format(fname))
            allUsers = pd.concat([allUsers, tDf['User']])

In [57]:
@numba.jit
def clean_files():
    userData = pd.read_csv(os.getcwd() + "/data/profiledata/userData.csv", low_memory=False)
    userTweets = pd.read_csv(os.getcwd() + "/data/profiledata/userTweets.csv", low_memory=False)
    
    userTweets = userTweets.rename(columns={'User': 'username'})
    
    merged = pd.merge(userData, userTweets, how='inner', on=['username'])    
    newuserData = merged[userData.columns]
    newuserData = newuserData.set_index('username').drop_duplicates().reset_index()
    
    newuserTweets = merged[userTweets.columns]
    newuserTweets = newuserTweets.rename(columns={'username': 'User'})
    
    newuserTweets = newuserTweets.set_index(['User', 'ID']).drop_duplicates().reset_index()
    
    newuserData.to_csv(os.getcwd() + "/data/profiledata/userData.csv", index=None)
    newuserTweets.to_csv(os.getcwd() + "/data/profiledata/userTweets.csv", index=None)
    newuserData['username'].to_csv(os.getcwd() + "/data/profiledata/extractedUsers.csv", index=None)

In [58]:
clean_files()

/home/shresthanikesh23/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,1,2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [54]:
def age_to_created():
    userData = pd.read_csv('/data/profiledata/userData.csv')
    userData['age'] = pd.to_numeric(userData['age'], errors='coerce').fillna(0).astype(int)
    
    userData.insert(1, 'created', 0)
    userData['created'] = userData['age'].apply(lambda x: (datetime.now() - timedelta(days=int(x))).strftime("%Y-%m-%d"))
    userData = userData.drop('age', axis=1)
    userData = userData.drop('tweets', axis=1)
    return userData

# userData = age_to_created()
# userData.to_csv('data/profiledata/userData.csv', index=None)